In [1]:
# Import the required libraries
from selenium import webdriver
import pandas as pd
from tqdm import tqdm
import numpy as np
import os

In [47]:
# Start the chrome driver, configuring this on the local requires
# some time. See this for installation
# Note: this would run chromium not chrome although simply find command to install and run chrome
# https://stackoverflow.com/questions/22476112/using-chromedriver-with-selenium-python-ubuntu
driver = webdriver.Chrome('/home/nitin/Projects/Web-Scrapper-Selenium/chromedriver')

# Selenium and driver
Using selenium in chrome driver simply means we are going to run the chrome as a user using selenium. So we have functions to click on what we want and go through the browser as a user would. We have the added benefit that we can extract the tables in the webpage using selenium as well. This would be used in out task here.

## Steps:
1. Get to the main page using link and issueing command driver.get()
2. Fill the form. To fill the form you have to click on the radio buttons. We have to this activity through selenium as well. First we have to get that radio button we want to click upon as a selenium element. Use the complete xpath to locate the html element you want to click. To get the complete xpath of any element in the browser follow [this link](https://www.google.com/search?q=how+to+get+complete+xpath+in+chrome&oq=get+complete+xpath+&aqs=chrome.1.69i57j0l2.5527j0j4&sourceid=chrome&ie=UTF-8#kpvalbx=_tauiXvbSGZ6a4-EPwa-FmAc42). Once you have the xpath use the driver.find_element_by_xpath() function which returns the required html element. Once you have the selenium html element, do .click() to click on the radio button.
3. Once form is filled by clicking on required radio buttons, use the same technique to get the selenium element of submit button. Then do .click to open the search results.
4. Search results generally comprise of different pages, we are going to use the following process to get each page.
5. We will extract the entire page table, then we will click on the next page using technique discussed above.
6. We iterate this process and append the array in one dataframe, till all the pages are extracted.
7. Then we get the data in csv using .to_csv() method in pandas dataframe.

In [50]:
# To get the main page
driver.get('https://pfaf.org/user/Default.aspx')

In [52]:
# xpath for A //*[@id="ctl00_ContentPlaceHolder1_HyperLink1"]
element = driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_HyperLink1"]')
element.click()

In [6]:
rows = len(driver.find_elements_by_xpath("/html/body/form/div[7]/table/tbody/tr[2]/td/div[1]/table/tbody/tr"))
cols = len(driver.find_elements_by_xpath("/html/body/form/div[7]/table/tbody/tr[2]/td/div[1]/table/tbody/tr[1]/th"))
print(rows, cols)

102 12


In [7]:
main_list = []
for r in range(2, rows+1):
    row_list = []
    for c in range(1, cols+1):
        value = driver.find_element_by_xpath("/html/body/form/div[7]/table/tbody/tr[2]/td/div[1]/table/tbody/tr["+str(r)+"]/td["+str(c)+"]").text
        row_list.append(value)
    main_list.append(row_list)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/form/div[7]/table/tbody/tr[2]/td/div[1]/table/tbody/tr[102]/td[2]"}
  (Session info: chrome=78.0.3904.87)


In [11]:
headers = []
for c in range(1,13):
        value = driver.find_element_by_xpath("/html/body/form/div[7]/table/tbody/tr[2]/td/div[1]/table/tbody/tr[1]/th["+str(c)+"]").text
        headers.append(value)

In [12]:
headers

['Latin Name',
 'Common Name',
 'Habit',
 'Height',
 'Hardiness',
 'Growth',
 'Soil',
 'Shade',
 'Moisture',
 'Edible',
 'Medicinal',
 'Other']

In [13]:
import pandas as pd
df = pd.DataFrame(main_list, columns = headers)
df.head()

,Latin Name,Common Name,Habit,Height,Hardiness,Growth,Soil,Shade,Moisture,Edible,Medicinal,Other
0,Abelia triflora,Indian Abelia,Shrub,3.5,5-9,M,LM,SN,DM,0,0,1
1,Abelmoschus esculentus,Okra,Annual,1.0,5-11,,LMH,N,M,4,3,2
2,Abelmoschus manihot,Aibika,Perennial,3.0,8-11,F,LMH,N,M,4,1,2
3,Abelmoschus moschatus,"Musk Mallow,Musk Okra",Perennial,2.0,8-11,F,LMH,N,M,2,3,
4,Abies alba,"Silver Fir, Christmas Tree Fir, European Silve...",Tree,45.0,5-8,F,LMH,FSN,M,2,3,4


In [14]:
df.tail()

,Latin Name,Common Name,Habit,Height,Hardiness,Growth,Soil,Shade,Moisture,Edible,Medicinal,Other
95,Acer pectinatum,Maple,Tree,10.0,3-6,S,LMH,SN,M,0,0,
96,Acer pensylvanicum,"Moosewood, Striped maple, Moosewood, Pennsylva...",Tree,4.5,3-7,,LMH,N,M,0,1,
97,Acer platanoides,"Norway Maple, Harlequin Maple",Tree,21.0,3-7,F,LMH,SN,M,2,0,3
98,Acer pseudoplatanus,"Sycamore, Great Maple, Scottish Maple, Planetr...",Tree,30.0,4-7,F,LMH,SN,M,2,1,4
99,Acer rubrum,"Red Maple, Drummond's maple, Swamp Maple",Tree,30.0,4-10,F,LMH,SN,M,3,1,3


In [ ]:
# saving data


In [4]:
# Get the selenium element to click on the salinity radio button
salinity_element = driver.find_element_by_xpath("/html/body/form/div[9]/div[2]/table/tbody/tr[52]/td[2]/div/table/tbody/tr/td[6]/input")

In [5]:
# Click on the radio button to select the salinity
salinity_element.click()

In [6]:
# Get the submit html element and click
submit_button = driver.find_element_by_xpath("/html/body/form/div[9]/div[3]/input[1]")
submit_button.click()

In [39]:
# /html/body/form/div[7]/table/tbody/tr[2]/td
# //*[@id="ctl00_ContentPlaceHolder1_gvresults"]
# /html/body/form/div[7]/table/tbody/tr[2]/td/div[1]/table/tbody/
# //*[@id="ctl00_ContentPlaceHolder1_gvresults"]
# Function to extract one table fromt the browser
# //*[@id="ctl00_ContentPlaceHolder1_tblpntcnt"]
# /html/body/form/div[7]/table
# /html/body/form/div[7]/table/tbody/tr[2]/td/div[1]/table/tbody/tr[2]/td

def scrap(driver):
    table = []
    names = driver.find_elements_by_xpath('/html/body/form/div[7]/table/tbody/tr[2]/td/div[1]/table/tbody/tr')
    print(len(names))
    for i in tqdm(range(len(names))):
        if i!=0:
            a = names[i].find_elements_by_tag_name('td')
        else:
            a = names[i].find_elements_by_tag_name('th')
        res = []
        for i in a:
            res.append(i.text)
        table.append(res)
    return table
table = scrap(driver)
table

  1%|          | 1/102 [00:00<00:17,  5.62it/s]

102


100%|██████████| 102/102 [00:19<00:00,  5.36it/s]


[['Latin Name',
  'Common Name',
  'Habit',
  'Height',
  'Hardiness',
  'Growth',
  'Soil',
  'Shade',
  'Moisture',
  'Edible',
  'Medicinal',
  'Other'],
 ['Abelia triflora',
  'Indian Abelia',
  'Shrub',
  '3.5',
  '5-9',
  'M',
  'LM',
  'SN',
  'DM',
  '0',
  '0',
  '1'],
 ['Abelmoschus esculentus',
  'Okra',
  'Annual',
  '1.0',
  '5-11',
  ' ',
  'LMH',
  'N',
  'M',
  '4',
  '3',
  '2'],
 ['Abelmoschus manihot',
  'Aibika',
  'Perennial',
  '3.0',
  '8-11',
  'F',
  'LMH',
  'N',
  'M',
  '4',
  '1',
  '2'],
 ['Abelmoschus moschatus',
  'Musk Mallow,Musk Okra',
  'Perennial',
  '2.0',
  '8-11',
  'F',
  'LMH',
  'N',
  'M',
  '2',
  '3',
  ' '],
 ['Abies alba',
  'Silver Fir, Christmas Tree Fir, European Silver Fir, Silver',
  'Tree',
  '45.0',
  '5-8',
  'F',
  'LMH',
  'FSN',
  'M',
  '2',
  '3',
  '4'],
 ['Abies amabilis',
  'Red Fir,Pacific silver fir',
  'Tree',
  '30.0',
  '4-8',
  'S',
  'LMH',
  'FSN',
  'M',
  '1',
  '2',
  ' '],
 ['Abies balsamea',
  'Balsam Fir',
  

In [54]:
# Get the tables from each page
# /html/body/form/div[7]/table/tbody/tr[2]/td/div[1]/table/tbody/tr[102]/td/table/tbody/tr/td[2]/a

res = []
for i in range(4):
    table = scrap(driver)
    df_table = pd.DataFrame(table[1:], columns=table[0])
    res.append(df_table)
    
    if i!=3:
#         next_page = driver.find_element_by_xpath("/html/body/form/div[7]/table/tbody/tr[2]/td/div[3]/table/tbody/tr[102]/td/table/tbody/tr/td["+str(i+2)+"]")
        next_page = driver.find_element_by_xpath("/html/body/form/div[7]/table/tbody/tr[2]/td/div[1]/table/tbody/tr[102]/td/table/tbody/tr/td["+str(i+2)+"]")
        
        next_page.click()

  0%|          | 0/102 [00:00<?, ?it/s]

102


  0%|          | 0/102 [00:00<?, ?it/s]

102


  0%|          | 0/102 [00:00<?, ?it/s]

102


  0%|          | 0/102 [00:00<?, ?it/s]

102


100%|██████████| 102/102 [00:22<00:00,  4.59it/s]


In [55]:
res = pd.concat(res)

In [56]:
res

,Latin Name,Common Name,Habit,Height,Hardiness,Growth,Soil,Shade,Moisture,Edible,Medicinal,Other
0,Abelia triflora,Indian Abelia,Shrub,3.5,5-9,M,LM,SN,DM,0,0,1
1,Abelmoschus esculentus,Okra,Annual,1.0,5-11,,LMH,N,M,4,3,2
2,Abelmoschus manihot,Aibika,Perennial,3.0,8-11,F,LMH,N,M,4,1,2
3,Abelmoschus moschatus,"Musk Mallow,Musk Okra",Perennial,2.0,8-11,F,LMH,N,M,2,3,
4,Abies alba,"Silver Fir, Christmas Tree Fir, European Silve...",Tree,45.0,5-8,F,LMH,FSN,M,2,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Allium przewalskianum,,Bulb,0.3,7-10,,L,N,DM,3,2,
97,Allium ramosum,Chinese Chives,Bulb,0.5,6-9,,LM,N,DM,4,2,
98,Allium roseum,Rosy Garlic,Bulb,0.4,7-10,,LM,N,DM,3,2,
99,Allium rubellum,,Bulb,0.6,-,,LM,N,DM,3,2,


In [10]:
res

,LatinName,CommonName,pH,EdibilityRating,MedicinalRating,OtherUsesRating
0,Acacia auriculiformis,"Ear-Pod Wattle, Black Acacia, Earleaf, Black w...",AN,1,0,4
1,Acacia farnesiana,"Sweet Acacia, Perfume Acacia, Huisache",ANB,2,2,
2,Acacia longifolia,"Sydney Golden Wattle, Acacia",ANB,3,0,
3,Acacia mucronata,Narrow-Leaf Wattle,ANB,2,0,
4,Acacia retinodes,"Swamp Wattle, Water wattle",ANB,2,0,
...,...,...,...,...,...,...
95,Caesalpinia echinata,"Pau Brasil, Brazil Wood, Indian Savin",NB,0,2,4
96,Calandrinia balonensis,,ANB,2,0,
97,Calandrinia polyandra,Parakeelya,ANB,2,0,
98,Calandrinia remota,,ANB,2,0,


In [11]:
# Save the dataframe as csv
res.to_csv("salinity.csv")

In [12]:
# close the driver
driver.close()